# Loading the required Libraries

In [8]:
from pathlib import Path
import numpy as np
import cv2
import pickle

## Defining the functions

In [9]:
def read_rgb_txt(path):
    with open(path, "r") as f:
        lines = f.readlines()

    n, m = map(int, lines[0].strip().split())

    data = [
        [list(map(int, pixel.split(","))) for pixel in line.strip().split()]
        for line in lines[1:]
    ]

    img = np.array(data, dtype=np.uint8)

    return img

def load_jpg_image(file_path):
    img = cv2.imread(str(file_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def read_output_txt(file_path):
    with open(file_path, 'r') as f:
        data = list(f.read())
    return data

def get_consecutive_zero_pairs(arr):
    pairs = []
    start_index = None

    for i in range(len(arr)):
        if arr[i] == 0 and start_index is None:
            start_index = i
        elif arr[i] != 0 and start_index is not None:
            pairs.append((start_index, i - 1))
            start_index = None

    # Check if the last element was part of a consecutive sequence
    if start_index is not None:
        pairs.append((start_index, len(arr) - 1))

    return pairs

def get_cropped_images(file_number):
    input_folder = Path.cwd().parent / "data" / "input"
    input_file = input_folder / f"input{file_number}.jpg"
    
    img = load_jpg_image(input_file)

    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    _, bw_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY)

    bw_img = (bw_img > 127).astype(int)

    min_col = np.min(bw_img, axis=0)
    consecutive_zero_pairs = get_consecutive_zero_pairs(min_col)

    min_row = np.min(bw_img, axis=1)
    consecutive_zero_pairs_row = get_consecutive_zero_pairs(min_row)

    start_row, end_row = consecutive_zero_pairs_row[0]

    all_images = []

    for i in range(5):
        start_col, end_col = consecutive_zero_pairs[i]
        
        cropped_img = bw_img[start_row:end_row+1, start_col:end_col+1]

        # If cropped image is not 10x8 then resize it to 10x8
        if cropped_img.shape != (10, 8):
            cropped_img = cv2.resize(cropped_img, (8, 10), interpolation=cv2.INTER_NEAREST)

        all_images.append(cropped_img)
    return all_images

def get_output(file_number):
    output_folder = Path.cwd().parent / "data" / "output"
    output_file = output_folder / f"output{file_number}.txt"
    output = read_output_txt(output_file)
    return output

def main(file_number):
    all_images = get_cropped_images(file_number)
    outputs = get_output(file_number)
    response = {}
    for output, img in zip(outputs, all_images):
        if output not in response:
            response[output] = [img]
        else:
            response[output].append(img)

    return response

In [10]:
# Create list of file numbers from 00 to 24
file_numbers = [f"{i:02d}" for i in range(25)]
training_data = file_numbers[:20] + ["24"]
test_data = file_numbers[20:-1] + ["100"] # Add "100" to test_data

all_training = {}
for file_number in training_data:
    response = main(file_number)
    for key in response:
        if key in all_training:
            all_training[key] = all_training[key] + response[key]
        else:
            all_training[key] = response[key]

all_training_mean = {}
for key in all_training:
    # Compute mean of all images in all_training[key] where all_training[key] is a list of 10x8 array
    all_training_mean[key] = np.mean(all_training[key], axis=0)

In [11]:
# Check that all capital letters and digits are present in all_training_mean (A-Z and 0-9)
characters = [chr(i) for i in range(65, 91)] + [str(i) for i in range(10)]
print(characters)

for character in characters:
    if character not in all_training_mean:
        print(f"{character} is missing in all_training_mean")


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [12]:
test_data

['20', '21', '22', '23', '100']

In [13]:
for test_name in test_data:
    
    test_images = get_cropped_images(test_name)
    test_output = get_output(test_name)

    for idx, test_image in enumerate(test_images):
        #compute correlation between test_image and all_training_mean for each key in all_training_mean
        correlations = {}
        for key in all_training_mean:
            correlation = np.corrcoef(test_image.flatten(), all_training_mean[key].flatten())[0, 1]
            correlations[key] = correlation
        #print highest correlation key
        predicted_key = max(correlations, key=correlations.get)
        print(f"Predicted: {predicted_key}, Actual: {test_output[idx]}, {predicted_key == test_output[idx]}")

Predicted: Z, Actual: Z, True
Predicted: 9, Actual: 9, True
Predicted: 7, Actual: 7, True
Predicted: M, Actual: M, True
Predicted: E, Actual: E, True
Predicted: C, Actual: C, True
Predicted: L, Actual: L, True
Predicted: 6, Actual: 6, True
Predicted: 9, Actual: 9, True
Predicted: V, Actual: V, True
Predicted: H, Actual: H, True
Predicted: C, Actual: C, True
Predicted: E, Actual: E, True
Predicted: 9, Actual: 9, True
Predicted: 1, Actual: 1, True
Predicted: W, Actual: W, True
Predicted: E, Actual: E, True
Predicted: L, Actual: L, True
Predicted: X, Actual: X, True
Predicted: V, Actual: V, True
Predicted: Y, Actual: Y, True
Predicted: M, Actual: M, True
Predicted: B, Actual: B, True
Predicted: 1, Actual: 1, True
Predicted: Q, Actual: Q, True


In [14]:
# Save all_training_mean to a pickle file
model_path = Path.cwd().parent / "model"
model_path.mkdir(exist_ok=True)

with open(model_path / 'model.pkl', 'wb') as f:
    pickle.dump(all_training_mean, f)